In [1]:
from bs4 import BeautifulSoup 
from urllib.request import Request, urlopen

import pandas as pd
import re

In [2]:
# df = pd.read_csv('data/exam-WebParsing.csv')       
df = pd.read_csv('data/chicagomag_info.csv', index_col='Rank') 
df.head()

,Unnamed: 0,Cafe,Menu,URL
Rank,,,,
1,0,Old Oak Tap,BLT,http://www.chicagomag.com/Chicago-Magazine/Nov...
2,1,Au Cheval,Fried Bologna,http://www.chicagomag.com/Chicago-Magazine/Nov...
3,2,Xoco,Woodland Mushroom,http://www.chicagomag.com/Chicago-Magazine/Nov...
4,3,Al’s Deli,Roast Beef,http://www.chicagomag.com/Chicago-Magazine/Nov...
5,4,Publican Quality Meats,PB&L,http://www.chicagomag.com/Chicago-Magazine/Nov...


In [3]:
# index_col='Unnamed: 0'') 삭제
del df['Unnamed: 0']
df.head()

,Cafe,Menu,URL
Rank,,,
1,Old Oak Tap,BLT,http://www.chicagomag.com/Chicago-Magazine/Nov...
2,Au Cheval,Fried Bologna,http://www.chicagomag.com/Chicago-Magazine/Nov...
3,Xoco,Woodland Mushroom,http://www.chicagomag.com/Chicago-Magazine/Nov...
4,Al’s Deli,Roast Beef,http://www.chicagomag.com/Chicago-Magazine/Nov...
5,Publican Quality Meats,PB&L,http://www.chicagomag.com/Chicago-Magazine/Nov...


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 50 entries, 1 to 50
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Cafe    50 non-null     object
 1   Menu    50 non-null     object
 2   URL     50 non-null     object
dtypes: object(3)
memory usage: 1.6+ KB


In [5]:
df['Menu'][1]

'BLT'

In [6]:
df['URL'][1]

'http://www.chicagomag.com/Chicago-Magazine/November-2012/Best-Sandwiches-in-Chicago-Old-Oak-Tap-BLT/'

### # 첫번째 URL 페이지 크롤링하기

In [7]:
def get_read_web(url):    
    req = Request(url, headers={'User-Agent': 'Mozilla/5.0'})
    html = urlopen(req).read()
    soup = BeautifulSoup(html, "lxml")    
    return soup

In [8]:
soup = get_read_web(df['URL'][1])
soup.title

<title>1. Old Oak Tap BLT – Chicago Magazine</title>

In [9]:
soup.find('p', 'addy')

<p class="addy">
<em>$10. 2109 W. Chicago Ave., 773-772-0406, <a href="http://www.theoldoaktap.com/">theoldoaktap.com</a></em></p>

In [10]:
taginfo = soup.find('p', 'addy').get_text()
taginfo

'\n$10. 2109 W. Chicago Ave., 773-772-0406, theoldoaktap.com'

In [11]:
taginfo = taginfo[taginfo.find('$'):]
taginfo

'$10. 2109 W. Chicago Ave., 773-772-0406, theoldoaktap.com'

In [12]:
price = re.search('\$\d+\.(\d+)?', taginfo).group()
price

'$10.'

In [13]:
address = '보류'

In [14]:
phone = re.search('\d{3}[-]\d{3}[-]\d{4}', taginfo)
if phone is not None:
    phone = phone.group()
phone

'773-772-0406'

In [15]:
infolist = taginfo.split(', ')
if infolist[-1].endswith('.com') or infolist[-1].endswith('.net'):
    homepage = infolist[-1]
else:
    homepage = 'No Homepage'
homepage

'theoldoaktap.com'

In [16]:
start = len(price)
if phone is None:
    phone = 'No Contact'
    end = taginfo.find(homepage)
else:
    end = taginfo.find(phone)
    
address = taginfo[start:end]
address

' 2109 W. Chicago Ave., '

### 상위 Top3 페이지에서 가격과 주소정보만 읽어오기

In [ ]:
Price    = []
Address  = []
Phone    = []
Homepage = []

for n in [1,2,3,4,5]:   
    # html = urlopen(df['URL'][n])
    # soup = BeautifulSoup(html, 'lxml')
    soup = get_read_web(df['URL'][n])
    
    taginfo = soup.find('p', 'addy').get_text()
    taginfo = taginfo[taginfo.find('$'):]
    
    price = re.search('\$\d+\.(\d+)?', taginfo).group()
    
    address = '일단보류'    # Street, City
    
    phone = re.search('\d{3}[-]\d{3}[-]\d{4}', taginfo)
    if phone is not None:
        phone = phone.group()    
        
    infolist = taginfo.split(', ')
    if infolist[-1].endswith('.com') or infolist[-1].endswith('.net'):
        homepage = infolist[-1]
    else:
        homepage = 'No Homepage'

    start = len(price)
    if phone is None:
        phone = 'No Contact'
        end = taginfo.find(homepage)
    else:
        end = taginfo.find(phone)

    address = taginfo[start:end] 

    Price.append(price)
    Address.append(address)
    Phone.append(phone)
    Homepage.append(homepage)
    
    print(n)